In [1]:
# Перечень команд, которые необходимо выполнить перед запуском jupyter notebook для включения решателей:
# export PATH=$PATH:/Library/Frameworks/GAMS.framework/Versions/47/Resources/
# export PATH=$PATH:/Library/Frameworks/ampl_macos64/

# Доступные решатели:
# _neos              [-] внутренняя ошибка связанная с работой со строками
# _mock_cbc          [-] попытка доступа к несуществующему полю _problem_files у ConcreteModel
# glpk               [-] не работает с нелинейными выражениями в целевой функции
# _glpk_shell        [-] не работает с нелинейными выражениями в целевой функции
# _mock_glpk         [-] попытка доступа к несуществующему полю _problem_files у ConcreteModel
# _mock_cplex        [-] не работает с нелинейными выражениями в целевой функции
# gurobi_direct      [-] не работает с нелинейными выражениями в целевой функции
# gurobi             [-] не работает с нелинейными выражениями в целевой функции
# _gurobi_shell      [-] не работает с нелинейными выражениями в целевой функции
# baron              [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# xpress             [-] не работает с нелинейными выражениями в целевой функции
# ipopt              [+] не работает с целыми значениями
# gurobi_persistent  [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gams               [+]
# _gams_shell        [+]
# xpress_direct      [-] не работает с нелинейными выражениями в целевой функции
# xpress_persistent  [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mpec_nlp           [+] не работает с целыми значениями
# mpec_minlp         [-] не работает с нелинейными выражениями в целевой функции
# appsi_gurobi       [-] не работает с нелинейными выражениями в целевой функции
# gdpopt             [-] требуется дополнительное указание алгоритма через аргумент метода solve         
# gdpopt.gloa        [-] ошибка инициализации решателя
# gdpopt.lbb         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.loa         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.ric         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.enumerate   [-] не работает с нелинейными выражениями в целевой функции
# mindtpy            [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.oa         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.ecp        [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.goa        [-] внутренняя ошибка работы решателя
# mindtpy.fp         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# multistart         [+] не работает с целыми значениями
# ipopt_v2           [+] не работает с целыми значениями
# gurobi_v2          [-] не работает с нелинейными выражениями в целевой функции
# gurobi_direct_v2   [-] не работает с нелинейными выражениями в целевой функции
# trustregion        [-] для инициализации требуется дополнительный аргумент degrees_of_freedom_variables
# ampl               [-] внутренняя ошибка работы решателя

In [2]:
import pyomo.environ as pyo
import numpy as np

In [3]:
M = 1000
e = np.e

In [4]:
# Инициализация модели
model = pyo.ConcreteModel(name = 'Optimal decomposition')

In [5]:
# Ограничения на сумму элементов в строках
model.constraints = pyo.ConstraintList()

In [6]:
model.f = pyo.VarList(domain=pyo.Binary)

In [7]:
# check_include = lambda x: (e ** (x * M) - e ** -(x * M)) / (e ** (x * M) + e ** -(x * M))
def check_include(x):
    for el in x:
        f = model.f.add()
        model.constraints.add((M * f - el >= 1))
        model.constraints.add((f - M * el <= 0))
    return x

In [8]:
# check_implemented = lambda x: e ** ((x - 1) * M)
def check_implemented(x):
    for el in x:
        f = model.f.add()
        model.constraints.add((M * f - (el - 1) >= 0))
        model.constraints.add((f - M * el <= 0))
    return x

In [9]:
calculate_dependencies = lambda i, F: np.dot(F, Eff) if i == 0 else np.dot(calculate_dependencies(i - 1, F), Eff)

In [10]:
# Матрица трассируемости требований к ПО на файлы исходного кода
Erf = np.array([[  1,   0,   0,   0],  # Требование № 1 реализовано в файле № 1
                [0.5, 0.5,   0,   0],  # Требование № 2 реализовано в файлах № 1 и 2
                [  0, 0.5, 0.5,   0],  # Требования № 3 реализовано в файлах № 2 и 3
                [  0,   0, 0.5, 0.5],  # Требование № 4 реализовано в файлах № 3 и 4
                [  0,   0,   0,   1]]) # Требовнаие № 5 реализовано в файле № 4

In [11]:
# Матрица зависимостей между файлами исходного кода
Eff = np.array([[0, 1, 0, 0],  # Файл № 1 имеет зависимость на файл № 2
                [0, 0, 0, 0],  # Файл № 2 не имеет зависимостей на другие файлы
                [0, 0, 0, 0],  # Файл № 2 не имеет зависимостей на другие файлы
                [0, 0, 0, 0]]) # Файл № 2 не имеет зависимостей на другие файлы

In [12]:
# Матрица стоимостей вхождения требований в поставку
Err = np.array([
    [1, 0, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 1
    [0, 1, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 2
    [0, 0, 1, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 3
    [0, 0, 0, 1, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 4
    [0, 0, 0, 0, 1]  # Стоимость сопровождения функционала, если в поставку войдет требование № 5
])

In [13]:
# Число плагинов
k = 3

In [14]:
# Число требований(n) и число файлов исходного кода(m)
n, m = np.shape(Erf)

In [15]:
# Матрица распределения файлов по плагинам
# Необходимо определить оптимальное распределение
# Здесь указаны ограничения на максимальное и минимальное значение
# И на то, что элементы должны быть целыми числами
set_m = pyo.Set(initialize=range(m))
set_k = pyo.Set(initialize=range(k))
model.Efp = pyo.Var(set_m, set_k, domain=pyo.Binary)

# Efp = np.array([
#     [1, 0, 0],
#     [0, 1, 0],
#     [0, 1, 0],
#     [0, 0, 1]
# ])


In [16]:
for row in np.array(model.Efp):
    model.constraints.add((sum(row) == 1))

In [17]:
R = np.array([1, 0, 0, 0, 0])

In [18]:
F = np.dot(R, Erf)

In [19]:
files_count = len(F)

In [20]:
for i in range(files_count):
    dependencies = calculate_dependencies(i, F)
    F += dependencies

In [21]:
P = np.dot(F, model.Efp)

In [22]:
F_d = np.dot(model.Efp, P)

In [23]:
F_d = check_include(M * F_d)

In [24]:
R_d = np.dot(Erf, F_d)

In [25]:
R_d = check_implemented(R_d)

In [26]:
costs = np.dot(Err, R_d)

In [27]:
model.OBJ = pyo.Objective(expr = sum(costs), sense=pyo.minimize)

In [28]:
model.pprint()

2 Var Declarations
    Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :     1 : False :  True : Binary
        (0, 1) :     0 :  None :     1 : False :  True : Binary
        (0, 2) :     0 :  None :     1 : False :  True : Binary
        (1, 0) :     0 :  None :     1 : False :  True : Binary
        (1, 1) :     0 :  None :     1 : False :  True : Binary
        (1, 2) :     0 :  None :     1 : False :  True : Binary
        (2, 0) :     0 :  None :     1 : False :  True : Binary
        (2, 1) :     0 :  None :     1 : False :  True : Binary
        (2, 2) :     0 :  None :     1 : False :  True : Binary
        (3, 0) :     0 :  None :     1 : False :  True : Binary
        (3, 1) :     0 :  None :     1 : False :  True : Binary
        (3, 2) :     0 :  None :     1 : False :  True : Binary
    f : Size=9, Index={1, 2, 3, 4, 5, 6, 7, 8, 9}
        Key : Lower : Value : Upper : Fixed : Stale

In [29]:
def solve(solver_name):
    solver = pyo.SolverFactory(solver_name)
    instance = model.create_instance()
    result = solver.solve(instance)
    instance.Efp.display()

In [30]:
solve('gams')

model.name="Optimal decomposition";
    - termination condition: intermediateNonInteger
    - message from solver: Solver quit with a problem (see LST file)
Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value             : Upper : Fixed : Stale : Domain
    (0, 0) :     0 : 0.916239570809765 :     1 : False : False : Binary
    (0, 1) :     0 : 0.083760429190235 :     1 : False : False : Binary
    (0, 2) :     0 :               0.0 :     1 : False : False : Binary
    (1, 0) :     0 : 0.000924289718854 :     1 : False : False : Binary
    (1, 1) :     0 :               0.0 :     1 : False : False : Binary
    (1, 2) :     0 : 0.999075710281146 :     1 : False : False : Binary
    (2, 0) :     0 :               0.0 :     1 : False : False : Binary
    (2, 1) :     0 :               1.0 :     1 : False : False : Binary
    (2, 2) :     0 :               0.0 :     1 : False : False : Binary
    (3, 0) :     0 :               0.0 :     1 : False : False : Binary
    (3,

In [31]:
solve('ipopt')

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value                  : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :    0.08376043430166805 :     1 : False : False : Binary
    (0, 1) :     0 : -9.865538164155106e-09 :     1 : False : False : Binary
    (0, 2) :     0 :       0.91623957556387 :     1 : False : False : Binary
    (1, 0) :     0 : -9.986784294717397e-09 :     1 : False : False : Binary
    (1, 1) :     0 :     0.9990757209386824 :     1 : False : False : Binary
    (1, 2) :     0 :  0.0009242890481019187 :     1 : False : False : Binary
    (2, 0) :     0 :      1.000000009998811 :     1 : False : False : Binary
    (2, 1) :     0 :  -9.97687717628468e-09 :     1 : False : False : Binary
    (2, 2) :     0 :  -2.19336921914697e-11 :     1 : False : False : Binary
    (3, 0) :     0 :     1.0000000099998136 :     1 : False : False : Binary
    (3, 1) :     0 :  -9.96329987450023e-09 :     1 : False : False : Binary
    (3, 2) :     0 : -3.65137239

In [32]:
solve('mpec_nlp')

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value                  : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :    0.08376043430166805 :     1 : False : False : Binary
    (0, 1) :     0 : -9.865538164155106e-09 :     1 : False : False : Binary
    (0, 2) :     0 :       0.91623957556387 :     1 : False : False : Binary
    (1, 0) :     0 : -9.986784294717397e-09 :     1 : False : False : Binary
    (1, 1) :     0 :     0.9990757209386824 :     1 : False : False : Binary
    (1, 2) :     0 :  0.0009242890481019187 :     1 : False : False : Binary
    (2, 0) :     0 :      1.000000009998811 :     1 : False : False : Binary
    (2, 1) :     0 :  -9.97687717628468e-09 :     1 : False : False : Binary
    (2, 2) :     0 :  -2.19336921914697e-11 :     1 : False : False : Binary
    (3, 0) :     0 :     1.0000000099998136 :     1 : False : False : Binary
    (3, 1) :     0 :  -9.96329987450023e-09 :     1 : False : False : Binary
    (3, 2) :     0 : -3.65137239

In [33]:
solve('multistart')

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value                   : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :      0.0837604349915605 :     1 : False :  True : Binary
    (0, 1) :     0 :  -6.300550406876709e-09 :     1 : False :  True : Binary
    (0, 2) :     0 :        0.91623957130899 :     1 : False :  True : Binary
    (1, 0) :     0 :  -9.636399153599606e-09 :     1 : False :  True : Binary
    (1, 1) :     0 :      0.9990757174638105 :     1 : False :  True : Binary
    (1, 2) :     0 :   0.0009242921725886432 :     1 : False :  True : Binary
    (2, 0) :     0 :      1.0000000099175441 :     1 : False :  True : Binary
    (2, 1) :     0 :  -9.669165058764046e-09 :     1 : False :  True : Binary
    (2, 2) :     0 : -2.4837908265521705e-10 :     1 : False :  True : Binary
    (3, 0) :     0 :      1.0000000099445228 :     1 : False :  True : Binary
    (3, 1) :     0 :  -9.481950253937542e-09 :     1 : False :  True : Binary
    (3, 2) :     0 :

In [34]:
solve('ipopt_v2')

Efp : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value                  : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :    0.08376043430166805 :     1 : False : False : Binary
    (0, 1) :     0 : -9.865538164155106e-09 :     1 : False : False : Binary
    (0, 2) :     0 :       0.91623957556387 :     1 : False : False : Binary
    (1, 0) :     0 : -9.986784294717397e-09 :     1 : False : False : Binary
    (1, 1) :     0 :     0.9990757209386824 :     1 : False : False : Binary
    (1, 2) :     0 :  0.0009242890481019187 :     1 : False : False : Binary
    (2, 0) :     0 :      1.000000009998811 :     1 : False : False : Binary
    (2, 1) :     0 :  -9.97687717628468e-09 :     1 : False : False : Binary
    (2, 2) :     0 :  -2.19336921914697e-11 :     1 : False : False : Binary
    (3, 0) :     0 :     1.0000000099998136 :     1 : False : False : Binary
    (3, 1) :     0 :  -9.96329987450023e-09 :     1 : False : False : Binary
    (3, 2) :     0 : -3.65137239

In [35]:
from pyomo.contrib.latex_printer import latex_printer
pstr = latex_printer(model)